In [1]:
import pystac
from pathlib import Path
from shapely.geometry import Polygon, mapping

import pandas as pd
from rasterio.warp import transform_bounds, transform_geom

import geopandas as gpd
from datetime import datetime, timezone


In [2]:
catalog = pystac.Catalog(
    id='forecast-test',
    title="Landsat FOVs",
    description='This is a test of Landsat image forecasts',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [3]:
grid = gpd.read_file('./tmp/n_visits.geojson')
bbox = grid.total_bounds
print(grid.crs.to_string())
print(bbox)

EPSG:4326
[-81.41094255 -55.61183    -34.41094255  13.38817   ]


In [4]:
def get_bbox_and_footprint(bounds):

    # create the bounding box it will depend if it comes from rasterio or rioxarray
    bbox = bounds

    # create the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],
        [bbox[0], bbox[3]],
        [bbox[2], bbox[3]],
        [bbox[2], bbox[1]]
    ])

    return bbox, mapping(footprint)

In [5]:
get_bbox_and_footprint(grid.total_bounds)

(array([-81.41094255, -55.61183   , -34.41094255,  13.38817   ]),
 {'type': 'Polygon',
  'coordinates': (((-81.41094255239946, -55.61183),
    (-81.41094255239946, 13.38817),
    (-34.41094255239946, 13.38817),
    (-34.41094255239946, -55.61183),
    (-81.41094255239946, -55.61183)),)})

In [6]:
folder = Path('./tmp/')
file_list = list(folder.rglob('*fovs.geojson'))

for f in file_list:
    fovs = gpd.read_file(f)

In [7]:
fovs

,satellite,id,time,lonspan,color,geometry
0,LANDSAT 8,139084,2021-12-10T11:46:00,4.272820,#ff0000ff,"POLYGON ((-31.63275 -49.86709, -34.17002 -49.4..."
1,LANDSAT 8,139084,2021-12-10T11:47:00,4.741502,#ff0000ff,"POLYGON ((-33.10701 -53.40744, -35.84204 -52.9..."
2,LANDSAT 8,139084,2021-12-10T13:07:00,2.468940,#ff0000ff,"POLYGON ((-40.43158 14.63123, -42.11168 14.986..."
3,LANDSAT 8,139084,2021-12-10T13:08:00,2.435245,#ff0000ff,"POLYGON ((-41.24147 11.00629, -42.89776 11.359..."
4,LANDSAT 8,139084,2021-12-10T13:09:00,2.413504,#ff0000ff,"POLYGON ((-42.03415 7.37979, -43.67395 7.73140..."
...,...,...,...,...,...,...
158,LANDSAT 9,149260,2021-12-11T14:54:00,3.230943,#0000ffff,"POLYGON ((-75.33794 -36.99183, -77.39034 -36.6..."
159,LANDSAT 9,149260,2021-12-11T14:55:00,3.446691,#0000ffff,"POLYGON ((-76.38030 -40.57311, -78.53853 -40.1..."
160,LANDSAT 9,149260,2021-12-11T14:56:00,3.710260,#0000ffff,"POLYGON ((-77.50673 -44.14533, -79.79017 -43.7..."
161,LANDSAT 9,149260,2021-12-11T14:57:00,4.035385,#0000ffff,"POLYGON ((-78.74017 -47.70704, -81.17217 -47.2..."


In [8]:
for i in range(len(fovs[:2])):
    fov = fovs.iloc[i]
    print(fov.time)
    print(fov.geometry)
    print(fov.geometry.bounds)

2021-12-10T11:46:00
POLYGON ((-31.63274996712584 -49.86709042356495, -34.17002159242003 -49.427162002979486, -35.905569694783715 -53.06823318003563, -33.169066208952565 -53.54632528353186, -31.63274996712584 -49.86709042356495))
(-35.905569694783715, -53.54632528353186, -31.63274996712584, -49.427162002979486)
2021-12-10T11:47:00
POLYGON ((-33.107012329081755 -53.40743501572923, -35.84203515596111 -52.94304080525593, -37.84851385930238 -56.567571945195816, -34.87256932886828 -57.07761598060291, -33.107012329081755 -53.40743501572923))
(-37.84851385930238, -57.07761598060291, -33.107012329081755, -52.94304080525593)


In [9]:
fovs

,satellite,id,time,lonspan,color,geometry
0,LANDSAT 8,139084,2021-12-10T11:46:00,4.272820,#ff0000ff,"POLYGON ((-31.63275 -49.86709, -34.17002 -49.4..."
1,LANDSAT 8,139084,2021-12-10T11:47:00,4.741502,#ff0000ff,"POLYGON ((-33.10701 -53.40744, -35.84204 -52.9..."
2,LANDSAT 8,139084,2021-12-10T13:07:00,2.468940,#ff0000ff,"POLYGON ((-40.43158 14.63123, -42.11168 14.986..."
3,LANDSAT 8,139084,2021-12-10T13:08:00,2.435245,#ff0000ff,"POLYGON ((-41.24147 11.00629, -42.89776 11.359..."
4,LANDSAT 8,139084,2021-12-10T13:09:00,2.413504,#ff0000ff,"POLYGON ((-42.03415 7.37979, -43.67395 7.73140..."
...,...,...,...,...,...,...
158,LANDSAT 9,149260,2021-12-11T14:54:00,3.230943,#0000ffff,"POLYGON ((-75.33794 -36.99183, -77.39034 -36.6..."
159,LANDSAT 9,149260,2021-12-11T14:55:00,3.446691,#0000ffff,"POLYGON ((-76.38030 -40.57311, -78.53853 -40.1..."
160,LANDSAT 9,149260,2021-12-11T14:56:00,3.710260,#0000ffff,"POLYGON ((-77.50673 -44.14533, -79.79017 -43.7..."
161,LANDSAT 9,149260,2021-12-11T14:57:00,4.035385,#0000ffff,"POLYGON ((-78.74017 -47.70704, -81.17217 -47.2..."


In [10]:
for f in file_list:
    fovs = gpd.read_file(f)
    for i in range(len(fovs[:2])):
        
        fov = fovs.iloc[i]

        bounds = fov.geometry.bounds
        bbox, footprint = get_bbox_and_footprint(bounds)

        # Project to WGS84 to obtain in geometric coordinates
        geo_bounds = transform_bounds(fovs.crs.to_string(), 'EPSG:4326', *bbox)
        geo_footprint = transform_geom(fovs.crs.to_string(), 'EPSG:4326', footprint)

        dt = datetime.fromisoformat(fov.time)
        dtz = dt.astimezone(timezone.utc)

        item = pystac.Item(
            # id=idx,
            id = str(i),
            geometry=geo_footprint,
            bbox=geo_bounds,
            datetime = dtz,
            stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
            properties=dict(
                # tile=tile
            )
        )

        catalog.add_item(item)

print(len(list(catalog.get_items())))
catalog.describe()

2
* <Catalog id=forecast-test>
  * <Item id=0>
  * <Item id=1>


In [11]:
fov.time

'2021-12-10T11:47:00'

In [12]:
# ! git clone https://github.com/stac-utils/pystac.git

In [13]:
catalog.describe()

* <Catalog id=forecast-test>
  * <Item id=0>
  * <Item id=1>


In [14]:
catalog.normalize_hrefs('./stac_catalog')
catalog.save()

In [15]:
from pystac import Catalog, get_stac_version
root_catalog = Catalog.from_file('./stac_catalog/catalog.json')
print(f"ID: {root_catalog.id}")
print(f"Title: {root_catalog.title or 'N/A'}")
print(f"Description: {root_catalog.description or 'N/A'}")

ID: forecast-test
Title: Landsat FOVs
Description: This is a test of Landsat image forecasts


In [16]:
item = catalog.get_item("1", recursive=True)


In [17]:
item.geometry

{'type': 'Polygon',
 'coordinates': [[(-37.84851385930238, -57.07761598060291),
   (-37.84851385930238, -52.94304080525593),
   (-33.107012329081755, -52.94304080525593),
   (-33.107012329081755, -57.07761598060291),
   (-37.84851385930238, -57.07761598060291)]]}

In [18]:
print(item.collection_id)

None
